In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [2]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    #Copy input
    movie = dict(movie)
    alt_titles = {}
    alt_title_cols = ['Also known as','Arabic','Cantonese','Chinese',
                      'French', 'Hangul','Hebrew','Hepburn','Japanese','Literally',
                      'Mandarin','McCune–Reischauer','Original title','Polish',
                      'Revised Romanization','Romanized','Russian',
                      'Simplified','Traditional','Yiddish']
    for key in alt_title_cols:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    
    if len(alt_titles) < 0:
        movie['alt_titles'] = alt_titles
    
    #Merge equivalent cols
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Country of origin', 'Country')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Original language(s)', 'Language')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompany ', 'Production Company')
    change_column_name('Production company(s)', 'Production Company')
    change_column_name('Productioncompanies ', 'Production Company')
    change_column_name('Release date', 'Released')    

    return movie

In [38]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    file_dir = "C:/Users/eanma/Google Drive/Data Analytics/Module 8/Movie-ETL/"
    
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory = False)
    ratings = pd.read_csv(f'{file_dir}ratings.csv')

    # Open and read the Wikipedia data JSON file.
    with open(f'{file_dir}wikipedia-movies.json', mode = 'r') as file:
        wiki_movies_raw = json.load(file)
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw if(
                ('Director' in movie or 'Directed by' in movie)
                and 'imdb_link' in movie
                and 'No. of episodes' not in movie)]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]
    
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    imdb_id = []
    
    for i in np.arange(0, len(wiki_movies_df) - 1):
        try: 
            if wiki_movies_df.at[i, 'imdb_id'] in imdb_id:
            #https://stackoverflow.com/questions/16729574/how-to-get-a-value-from-a-cell-of-a-dataframe
                wiki_movies_df.drop(index = row.index)
                
            else:
                imdb_id.append(row[imdb_id].str.extract(r'tt\d{7}'))
            
        except: print('Error: row ' + str(i))

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [col for col in wiki_movies_df.columns if(
                            wiki_movies_df[col].isnull().sum() < .9 * len(wiki_movies_df))]
    
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.map(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*milli?on'
    
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\s*\d+\.?\d*\s*billi?on'

    # 12. Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan
    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis = 1, inplace = True)
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex = True)
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags = re.I)[0].apply(parse_dollars)
    wiki_movies_df.drop('Budget', axis = 1, inplace = True)
    
    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_movies_df['Released'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'

    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format = True)
    wiki_movies_df.drop('Released', axis = 1, inplace = True)
    
    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors = 'coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0] * 60 + row[1] if row[2] == 0 else row[2], axis = 1)
    wiki_movies_df.drop('Running time', axis = 1, inplace = True)
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    return wiki_movies_df, kaggle_metadata, ratings

In [39]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = "C:/Users/eanma/Google Drive/Data Analytics/Module 8/Movie-ETL/"
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [40]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

Error: row 0
Error: row 1
Error: row 2
Error: row 3
Error: row 4
Error: row 5
Error: row 6
Error: row 7
Error: row 8
Error: row 9
Error: row 10
Error: row 11
Error: row 12
Error: row 13
Error: row 14
Error: row 15
Error: row 16
Error: row 17
Error: row 18
Error: row 19
Error: row 20
Error: row 21
Error: row 22
Error: row 23
Error: row 24
Error: row 25
Error: row 26
Error: row 27
Error: row 28
Error: row 29
Error: row 30
Error: row 31
Error: row 32
Error: row 33
Error: row 34
Error: row 35
Error: row 36
Error: row 37
Error: row 38
Error: row 39
Error: row 40
Error: row 41
Error: row 42
Error: row 43
Error: row 44
Error: row 45
Error: row 46
Error: row 47
Error: row 48
Error: row 49
Error: row 50
Error: row 51
Error: row 52
Error: row 53
Error: row 54
Error: row 55
Error: row 56
Error: row 57
Error: row 58
Error: row 59
Error: row 60
Error: row 61
Error: row 62
Error: row 63
Error: row 64
Error: row 65
Error: row 66
Error: row 67
Error: row 68
Error: row 69
Error: row 70
Error: row 71
Er

Error: row 1667
Error: row 1668
Error: row 1669
Error: row 1670
Error: row 1671
Error: row 1672
Error: row 1673
Error: row 1674
Error: row 1675
Error: row 1676
Error: row 1677
Error: row 1678
Error: row 1679
Error: row 1680
Error: row 1681
Error: row 1682
Error: row 1683
Error: row 1684
Error: row 1685
Error: row 1686
Error: row 1687
Error: row 1688
Error: row 1689
Error: row 1690
Error: row 1691
Error: row 1692
Error: row 1693
Error: row 1694
Error: row 1695
Error: row 1696
Error: row 1697
Error: row 1698
Error: row 1699
Error: row 1700
Error: row 1701
Error: row 1702
Error: row 1703
Error: row 1704
Error: row 1705
Error: row 1706
Error: row 1707
Error: row 1708
Error: row 1709
Error: row 1710
Error: row 1711
Error: row 1712
Error: row 1713
Error: row 1714
Error: row 1715
Error: row 1716
Error: row 1717
Error: row 1718
Error: row 1719
Error: row 1720
Error: row 1721
Error: row 1722
Error: row 1723
Error: row 1724
Error: row 1725
Error: row 1726
Error: row 1727
Error: row 1728
Error: r

Error: row 3248
Error: row 3249
Error: row 3250
Error: row 3251
Error: row 3252
Error: row 3253
Error: row 3254
Error: row 3255
Error: row 3256
Error: row 3257
Error: row 3258
Error: row 3259
Error: row 3260
Error: row 3261
Error: row 3262
Error: row 3263
Error: row 3264
Error: row 3265
Error: row 3266
Error: row 3267
Error: row 3268
Error: row 3269
Error: row 3270
Error: row 3271
Error: row 3272
Error: row 3273
Error: row 3274
Error: row 3275
Error: row 3276
Error: row 3277
Error: row 3278
Error: row 3279
Error: row 3280
Error: row 3281
Error: row 3282
Error: row 3283
Error: row 3284
Error: row 3285
Error: row 3286
Error: row 3287
Error: row 3288
Error: row 3289
Error: row 3290
Error: row 3291
Error: row 3292
Error: row 3293
Error: row 3294
Error: row 3295
Error: row 3296
Error: row 3297
Error: row 3298
Error: row 3299
Error: row 3300
Error: row 3301
Error: row 3302
Error: row 3303
Error: row 3304
Error: row 3305
Error: row 3306
Error: row 3307
Error: row 3308
Error: row 3309
Error: r

Error: row 4989
Error: row 4990
Error: row 4991
Error: row 4992
Error: row 4993
Error: row 4994
Error: row 4995
Error: row 4996
Error: row 4997
Error: row 4998
Error: row 4999
Error: row 5000
Error: row 5001
Error: row 5002
Error: row 5003
Error: row 5004
Error: row 5005
Error: row 5006
Error: row 5007
Error: row 5008
Error: row 5009
Error: row 5010
Error: row 5011
Error: row 5012
Error: row 5013
Error: row 5014
Error: row 5015
Error: row 5016
Error: row 5017
Error: row 5018
Error: row 5019
Error: row 5020
Error: row 5021
Error: row 5022
Error: row 5023
Error: row 5024
Error: row 5025
Error: row 5026
Error: row 5027
Error: row 5028
Error: row 5029
Error: row 5030
Error: row 5031
Error: row 5032
Error: row 5033
Error: row 5034
Error: row 5035
Error: row 5036
Error: row 5037
Error: row 5038
Error: row 5039
Error: row 5040
Error: row 5041
Error: row 5042
Error: row 5043
Error: row 5044
Error: row 5045
Error: row 5046
Error: row 5047
Error: row 5048
Error: row 5049
Error: row 5050
Error: r

Error: row 6586
Error: row 6587
Error: row 6588
Error: row 6589
Error: row 6590
Error: row 6591
Error: row 6592
Error: row 6593
Error: row 6594
Error: row 6595
Error: row 6596
Error: row 6597
Error: row 6598
Error: row 6599
Error: row 6600
Error: row 6601
Error: row 6602
Error: row 6603
Error: row 6604
Error: row 6605
Error: row 6606
Error: row 6607
Error: row 6608
Error: row 6609
Error: row 6610
Error: row 6611
Error: row 6612
Error: row 6613
Error: row 6614
Error: row 6615
Error: row 6616
Error: row 6617
Error: row 6618
Error: row 6619
Error: row 6620
Error: row 6621
Error: row 6622
Error: row 6623
Error: row 6624
Error: row 6625
Error: row 6626
Error: row 6627
Error: row 6628
Error: row 6629
Error: row 6630
Error: row 6631
Error: row 6632
Error: row 6633
Error: row 6634
Error: row 6635
Error: row 6636
Error: row 6637
Error: row 6638
Error: row 6639
Error: row 6640
Error: row 6641
Error: row 6642
Error: row 6643
Error: row 6644
Error: row 6645
Error: row 6646
Error: row 6647
Error: r

In [41]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [42]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

,url,year,imdb_link,title,Screenplay by,Story by,Based on,Starring,Cinematography,Country,...,Distributor,Editor(s),Composer(s),Producer(s),Production Company,Written by,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,United States,...,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,NaN,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,United States,...,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,NaN,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,United States,...,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]",NaN,NaN,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,United States,...,Orion Pictures,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,NaN,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,US,...,Paramount Pictures,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,NaN,25000000.0,1990-12-19,95.0


In [43]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Cinematography',
 'Country',
 'Language',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production Company',
 'Written by',
 'box_office',
 'budget',
 'release_date',
 'running_time']